**Check System Config**

In [1]:
# Check system configuration
# Expects cuda 11.8, Python 3.10.11, Ubuntu 20.04.5, gcc 9.4.0
!nvcc --version
!python --version
!cat /etc/os-release | head -n2
!gcc --version

# If nvcc throws NameError, check if GPU is being used.
# If not, change device to GPU via Runtime -> Change runtime type

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Python 3.10.12
PRETTY_NAME="Ubuntu 22.04.3 LTS"
NAME="Ubuntu"
gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



**Get Code and Install Packages**

In [2]:
%%shell

# Clone SHAPY repo
git clone https://github.com/muelea/shapy.git
cd /content/shapy

Cloning into 'shapy'...
remote: Enumerating objects: 441, done.
remote: Counting objects: 100% (194/194), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 441 (delta 96), reused 122 (delta 69), pack-reused 247
Receiving objects: 100% (441/441), 13.64 MiB | 16.52 MiB/s, done.
Resolving deltas: 100% (109/109), done.


In [3]:
%%shell

# Install python dependencies
pip install chumpy imageio jpeg4py joblib kornia loguru matplotlib numpy open3d Pillow PyOpenGL pyrender scikit-image scikit-learn scipy smplx tqdm trimesh yacs fvcore nflows PyYAML
pip install omegaconf==2.0.6
pip install pytorch-lightning

# Install local dependencies
cd /content/shapy/attributes
pip install .

cd /content/shapy/mesh-mesh-intersection
export CUDA_SAMPLES_INC=$(pwd)/include
pip install -r requirements.txt
pip install .

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 664.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 694.6/694.6 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

**Download SMPL-X body model and SHAPY model and utilities**

In [4]:
%%shell
# Download SHAPY utils
# Please register at the SHAPY website: https://shapy.is.tue.mpg.de
# It'll ask for your username and password

cd /content/shapy/data

urle () { [[ "${1}" ]] || return 1; local LANG=C i x; for (( i = 0; i < ${#1}; i++ )); do x="${1:i:1}"; [[ "${x}" == [a-zA-Z0-9.~-] ]] && echo -n "${x}" || printf '%%%02X' "'${x}"; done; echo; }

#read -p "Username:" username robinvegoblin@gmail.com
#read -p "Password:" -s password Dp=qck_5#kaitqQ

username=$(urle 'robinvegoblin@gmail.com')
password=$(urle 'Dp=qck_5#kaitqQ')


# SHAPY
wget --post-data "username=$username&password=$password" 'https://download.is.tue.mpg.de/download.php?domain=shapy&resume=1&sfile=shapy_data.zip' -O 'shapy_data.zip' --no-check-certificate --continue
unzip shapy_data.zip
rm shapy_data.zip

--2024-05-31 12:07:10--  https://download.is.tue.mpg.de/download.php?domain=shapy&resume=1&sfile=shapy_data.zip
Resolving download.is.tue.mpg.de (download.is.tue.mpg.de)... 192.124.27.139
Connecting to download.is.tue.mpg.de (download.is.tue.mpg.de)|192.124.27.139|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: download.php?domain=shapy&sfile=shapy_data.zip&resume=1 [following]
--2024-05-31 12:07:11--  https://download.is.tue.mpg.de/download.php?domain=shapy&sfile=shapy_data.zip&resume=1
Reusing existing connection to download.is.tue.mpg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 2194922772 (2.0G) [application/octet-stream]
Saving to: ‘shapy_data.zip’

shapy_data.zip      100%[===================>]   2.04G  25.2MB/s    in 1m 44s  

2024-05-31 12:08:54 (20.2 MB/s) - ‘shapy_data.zip’ saved [2194922772/2194922772]

Archive:  shapy_data.zip
   creating: expose_release/
   creating: expose_release/utility_files/
   creating: expose_release/

In [5]:
# Download SMPL-X model
# You must register at the SMPL-X website: https://smpl-x.is.tue.mpg.de
# The script will ask for username and password
%%shell

cd /content/shapy/data

urle () { [[ "${1}" ]] || return 1; local LANG=C i x; for (( i = 0; i < ${#1}; i++ )); do x="${1:i:1}"; [[ "${x}" == [a-zA-Z0-9.~-] ]] && echo -n "${x}" || printf '%%%02X' "'${x}"; done; echo; }

#read -p "Username:" username
#read -p "Password:" -s password

username=$(urle 'robinvegoblin@gmail.com')
password=$(urle '~Rf4sAy]2SZL&d=')

# SMPL-X
wget --post-data "username=$username&password=$password" 'https://download.is.tue.mpg.de/download.php?domain=smplx&sfile=models_smplx_v1_1.zip&resume=1' -O 'models_smplx_v1_1.zip' --no-check-certificate --continue
unzip models_smplx_v1_1.zip
mv models body_models
rm models_smplx_v1_1.zip

--2024-05-31 12:09:54--  https://download.is.tue.mpg.de/download.php?domain=smplx&sfile=models_smplx_v1_1.zip&resume=1
Resolving download.is.tue.mpg.de (download.is.tue.mpg.de)... 192.124.27.139
Connecting to download.is.tue.mpg.de (download.is.tue.mpg.de)|192.124.27.139|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: download.php?domain=smplx&sfile=models_smplx_v1_1.zip&resume=1 [following]
--2024-05-31 12:09:55--  https://download.is.tue.mpg.de/download.php?domain=smplx&sfile=models_smplx_v1_1.zip&resume=1
Reusing existing connection to download.is.tue.mpg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 870108517 (830M) [application/octet-stream]
Saving to: ‘models_smplx_v1_1.zip’

models_smplx_v1_1.z 100%[===================>] 829.80M  26.7MB/s    in 32s     

2024-05-31 12:10:27 (25.7 MB/s) - ‘models_smplx_v1_1.zip’ saved [870108517/870108517]

Archive:  models_smplx_v1_1.zip
   creating: models/
   creating: models/smplx/
  inflating: 

**Quick Bug Fix**

Since we're using a different Python version in Colab, we must make small edit in /content/shapy/regressor/human_shape/models/backbone/resnet.py.
Open file and remove "model_urls" in line 13.

**Run Demo Code**

In [ ]:
!pip install chumpy imageio jpeg4py joblib kornia loguru matplotlib numpy open3d Pillow PyOpenGL pyrender scikit-image scikit-learn scipy smplx tqdm trimesh yacs fvcore nflows PyYAML

  Using cached chumpy-0.70.tar.gz (50 kB)
  Preparing metadata (setup.py) ... done
  Using cached jpeg4py-0.1.4.tar.gz (12 kB)
  Preparing metadata (setup.py) ... done
  Using cached kornia-0.7.2-py2.py3-none-any.whl (825 kB)
  Using cached loguru-0.7.2-py3-none-any.whl (62 kB)
  Using cached open3d-0.18.0-cp310-cp310-manylinux_2_27_x86_64.whl (399.7 MB)
  Using cached pyrender-0.1.45-py3-none-any.whl (1.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 694.6/694.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [6]:
#If import mesh_mesh_intersection throws an error, try exporting PYTHONPATH

import os
os.environ['PYTHONPATH'] += ':/content/shapy/attributes/'
os.environ['PYTHONPATH'] += ':/content/shapy/mesh-mesh-intersection/'
os.environ['PYTHONPATH'] += ':/usr/local'
import mesh_mesh_intersection

In [12]:
%%shell
cd /content/shapy/regressor
python demo.py --save-vis true --save-params true --show true --save-mesh true --split test --datasets openpose --output-folder samples/shapy_fit/ --exp-cfg configs/b2a_expose_hrnet_demo.yaml --exp-opts output_folder=../data/trained_models/shapy/SHAPY_A part_key=pose datasets.pose.openpose.data_folder=../samples datasets.pose.openpose.img_folder=images  datasets.pose.openpose.keyp_folder=openpose datasets.batch_size=1 datasets.pose_shape_ratio=1.0


2024-05-31 12:40:49.640 | INFO     | human_shape.models.build:build_model:18 - Going to build a: SMPLXRegressor
2024-05-31 12:40:49.641 | INFO     | human_shape.models.body_heads.body_heads:__init__:221 - Predict face: False
2024-05-31 12:40:49.642 | INFO     | human_shape.models.body_heads.body_heads:__init__:146 - Predict hands: False
2024-05-31 12:40:49.642 | INFO     | human_shape.models.common.iterative_regressor:__init__:53 - Pose last stage: True
2024-05-31 12:40:49.644 | INFO     | human_shape.models.body_models.body_models:__init__:545 - Loading model from: ../data/body_models/smplx/SMPLX_NEUTRAL.npz
2024-05-31 12:40:50.562 | INFO     | human_shape.models.body_models.body_models:__init__:173 - Use joint regressor: True
2024-05-31 12:40:50.562 | INFO     | human_shape.models.body_models.body_models:__init__:175 - Loading joint regressor from: ../data/expose_release/data/SMPLX_to_J14.pkl
2024-05-31 12:40:50.581 | INFO     | human_shape.models.body_heads.body_heads:_build_model:5

CalledProcessError: Command 'cd /content/shapy/regressor
python demo.py --save-vis true --save-params true --show true --save-mesh true --split test --datasets openpose --output-folder samples/shapy_fit/ --exp-cfg configs/b2a_expose_hrnet_demo.yaml --exp-opts output_folder=../data/trained_models/shapy/SHAPY_A part_key=pose datasets.pose.openpose.data_folder=../samples datasets.pose.openpose.img_folder=images  datasets.pose.openpose.keyp_folder=openpose datasets.batch_size=1 datasets.pose_shape_ratio=1.0
' returned non-zero exit status 1.

In [ ]:
%%shell
cd /content/shapy/measurements

# given SMPL betas compute anthropometric measurements
python virtual_measurements.py --input-folder ../samples/shapy_fit_for_virtual_measurements/ --output-folder=../samples/virtual_measurements/

Processing: img_00.npz
    Virtual measurements:     mass: 56.87 kg    height: 1.64 m    chest: 0.87 m    waist: 0.76 m    hips: 0.95 m


In [ ]:
%%shell

cd /content/shapy/attributes
ln -s /content/shapy/regressor/samples/shapy_fit /content/shapy/samples/shapy_fit # link examples (output from regrssor)

# given neutral betas estimate female attribute ratings
python demo.py --exp-cfg configs/s2a.yaml --exp-opts output_dir=../data/trained_models/b2a/polynomial/caesar-female_smplx-neutral-10betas ds_gender=female model_gender=neutral num_shape_comps=10

# given neutral betas estimate male attribute ratings
python demo.py --exp-cfg configs/s2a.yaml --exp-opts output_dir=../data/trained_models/b2a/polynomial/caesar-male_smplx-neutral-10betas ds_gender=male model_gender=neutral num_shape_comps=10

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/migration/migration.py:201: PossibleUserWarning: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an upgrade, which means we can't differentiate which of your checkpoint callbacks needs which states. At least one of your `ModelCheckpoint` callbacks will not be able to reload the state.
  rank_zero_warn(
Lightning automatically upgraded your loaded checkpoint from v1.3.0rc1 to v2.0.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../data/trained_models/b2a/polynomial/caesar-female_smplx-neutral-10betas/last.ckpt`
2023-05-22 18:38:51.560 | INFO     | attributes.utils.config:get_features_from_config:386 - {'big': True, 'broad_shoulders': True, 'large_breasts': True, 'long_legs': True, 'long_neck': True, 'long_torso': True, 'muscular': True, 'pear_shaped': T